# Tutorial 3: Simple Flows and Graph Basics

**Difficulty:** Beginner | **Time:** 20 minutes

## Learning Objectives

- Connect multiple nodes into a flow
- Understand edge creation with `>>` operator
- Create and run a graph
- Work with ExecutionContext
- Use `context.inputs` and return outputs correctly
- Pass data between nodes via outputs (not node state!)
- **NEW**: Use graph state for globally shared data across all nodes
- **Understand the three storage types**: outputs, node state, and graph state

## Real-World Use Case

Imagine you're building a data processing pipeline: first you collect data, then you clean it, transform it, analyze it, and finally generate a report. Each step depends on the previous one, and data needs to flow between them. Spark's graphs let you create these workflows as connected nodes, where each node does one specific job and passes results to the next **via outputs** (not state!).

Additionally, you might need to track metrics across the entire pipeline (like total items processed, error counts, or global flags). For this, Spark provides **graph state** - a globally shared state accessible to all nodes, perfect for coordination and aggregation.

In this tutorial, we'll build several example flows to understand how nodes work together in graphs, and we'll learn the critical distinction between the three storage types: outputs for sequential flow, node state for private data, and graph state for global coordination.

## Core Concepts

### Graph Architecture

A **Graph** in Spark is a collection of connected **Nodes** that process data in sequence:

```
Input Data → Node 1 → Node 2 → Node 3 → Output
             (outputs flow →)
```

### Key Components

1. **Node**: A processing unit that takes input and produces output
2. **Edge**: Connection between nodes that determines data flow
3. **Graph**: Container that orchestrates node execution
4. **ExecutionContext**: Runtime environment with inputs, node state, metadata, and graph state

### CRITICAL: The Actor Model - Three Types of Data Storage

**Spark implements the Actor Model with three distinct data storage mechanisms:**

1. **Outputs FLOW between nodes**: A node's return value becomes the next node's `context.inputs`
   - Use for: Data that's transformed/created by a node and consumed by specific successors
   - Flows: One node → Next node(s)

2. **Node State is NODE-LOCAL**: Each node's `context.state` is private and does NOT flow
   - Use for: A node's own persistent data (e.g., invocation counts, internal cache)
   - Flows: Nowhere, persists only for that node across invocations

3. **Graph State is GLOBALLY SHARED**: `context.graph_state` is accessible to ALL nodes
   - Use for: Shared counters, accumulated results, coordination flags, global configuration
   - Flows: Everywhere, accessible by all nodes in the graph

```python
# ❌ WRONG - Node state doesn't flow
async def process(self, context):
    context.state['data'] = value  # Next node won't see this!

# ✅ RIGHT - Outputs flow to next nodes
async def process(self, context):
    return {'data': value}  # Next node receives this as inputs

# ✅ RIGHT - Graph state is shared across all nodes
async def process(self, context):
    await context.graph_state.set('counter', 42)  # All nodes can access this!
```

### When to Use Each Storage Type

**Use Outputs when:**
- Data is transformed by a node and only its direct successors need it
- Each node in the pipeline adds/modifies the data
- The data flow is linear or tree-like

**Use Node State when:**
- Tracking a node's own invocation count, retry attempts, or internal cache
- The data is specific to one node's behavior across multiple executions
- Other nodes should NOT see this data

**Use Graph State when:**
- Multiple nodes need to read/write the same data (counters, flags, collections)
- Coordinating behavior across distant nodes in the graph
- Accumulating results that don't need to flow through every node
- Sharing configuration or context that all nodes need

### The `>>` Operator

The `>>` operator creates edges between nodes:

```python
node1 >> node2  # Output of node1 flows to node2's inputs
```

### Auto-Discovery

Spark automatically discovers all connected nodes when you create a graph with just the start node:

```python
graph = Graph(start=start_node)  # Auto-discovers all connected nodes
```

### Graph State Initialization

You can initialize graph state when creating a graph:

```python
# Empty graph state (default)
graph = Graph(start=start_node)

# With initial values
graph = Graph(start=start_node, initial_state={'counter': 0, 'results': []})
```

## Setup

Let's import the necessary classes and set up our environment.

In [ ]:
# Import core Spark classes
from spark.nodes import Node
from spark.graphs import Graph
from spark.utils import arun
import asyncio
import time
from typing import Dict, Any

## Example 1: Your First Simple Flow

Let's start with a basic 2-node flow: a data generator connected to a data processor.

In [ ]:
class DataGenerator(Node):
    """Generates sample data."""
    
    async def process(self, context):
        print("🏭 Generating data...")
        data = {
            'numbers': [1, 2, 3, 4, 5],
            'message': 'Hello from DataGenerator!'
        }
        print(f"✅ Generated: {data}")
        return data

class DataProcessor(Node):
    """Processes data from previous node."""
    
    async def process(self, context):
        # Access input from previous node
        input_data = context.inputs.content
        numbers = input_data.get('numbers', [])
        message = input_data.get('message', '')
        
        print(f"🔧 Processing: {message}")
        
        # Process the numbers (calculate sum and average)
        total = sum(numbers)
        average = total / len(numbers) if numbers else 0
        
        result = {
            'original_message': message,
            'numbers': numbers,
            'sum': total,
            'average': average,
            'processed_at': time.time()
        }
        
        print(f"✅ Processed result: sum={total}, average={average:.2f}")
        return result

# Create nodes
generator = DataGenerator()
processor = DataProcessor()

# Connect nodes with >> operator
generator >> processor

# Create graph with auto-discovery
graph = Graph(start=generator)

print("=== Example 1: Simple 2-Node Flow ===")
result = await graph.run()

print(f"\n🎯 Final result: {result.content}")

## Example 2: Multi-Step Pipeline

Let's build a longer pipeline with multiple processing steps.

In [ ]:
class TextCollector(Node):
    """Collects and prepares text for processing."""
    
    async def process(self, context):
        # Get initial text from context inputs
        text = context.inputs.content.get('text', 'Hello World')
        
        print(f"📝 Collecting text: '{text}'")
        
        return {
            'original_text': text,
            'word_count': len(text.split()),
            'char_count': len(text)
        }

class TextCleaner(Node):
    """Cleans and normalizes text."""
    
    async def process(self, context):
        data = context.inputs.content
        text = data.get('original_text', '')
        
        print(f"🧹 Cleaning text: '{text}'")
        
        # Clean the text
        cleaned = text.strip().lower()
        
        result = data.copy()
        result.update({
            'cleaned_text': cleaned,
            'has_numbers': any(char.isdigit() for char in text),
            'has_special_chars': any(not char.isalnum() and not char.isspace() for char in text)
        })
        
        return result

class TextAnalyzer(Node):
    """Analyzes text and extracts insights."""
    
    async def process(self, context):
        data = context.inputs.content
        text = data.get('cleaned_text', '')
        
        print(f"🔍 Analyzing text: '{text}'")
        
        # Analyze the text
        vowels = sum(1 for char in text if char in 'aeiou')
        consonants = sum(1 for char in text if char.isalpha() and char not in 'aeiou')
        
        # Extract words and analyze
        words = text.split()
        longest_word = max(words, key=len) if words else ''
        
        result = data.copy()
        result.update({
            'vowel_count': vowels,
            'consonant_count': consonants,
            'longest_word': longest_word,
            'analysis_complete': True
        })
        
        return result

class ReportGenerator(Node):
    """Generates a final report."""
    
    async def process(self, context):
        data = context.inputs.content
        
        print("📊 Generating report...")
        
        # Create a summary report
        report = f"""
=== Text Analysis Report ===
Original Text: {data.get('original_text', 'N/A')}
Cleaned Text: {data.get('cleaned_text', 'N/A')}
Word Count: {data.get('word_count', 0)}
Character Count: {data.get('char_count', 0)}
Vowels: {data.get('vowel_count', 0)}
Consonants: {data.get('consonant_count', 0)}
Longest Word: {data.get('longest_word', 'N/A')}
Contains Numbers: {data.get('has_numbers', False)}
Contains Special Chars: {data.get('has_special_chars', False)}
=============================
        """.strip()
        
        print(report)
        
        return {
            'report': report,
            'summary': {
                'original_length': data.get('char_count', 0),
                'words_processed': data.get('word_count', 0),
                'analysis_steps': 4
            }
        }

# Create the pipeline
collector = TextCollector()
cleaner = TextCleaner()
analyzer = TextAnalyzer()
reporter = ReportGenerator()

# Connect all nodes in sequence
collector >> cleaner >> analyzer >> reporter

# Create graph
pipeline = Graph(start=collector)

print("=== Example 2: Multi-Step Text Pipeline ===")
print("Processing: 'Hello Spark Framework 2024!'")

# Run with initial input
result = await pipeline.run({'text': 'Hello Spark Framework 2024!'})

print(f"\n🎯 Pipeline completed!")
print(f"Summary: {result.content['summary']}")

## Example 3: Understanding the Three Storage Types

This example demonstrates both INCORRECT and CORRECT patterns for each storage type.

In [ ]:
print("=== Example 3A: INCORRECT Pattern - Trying to Share State ===" )
print("⚠️  This demonstrates what NOT to do!\\n")

class StateSharer(Node):
    """Tries to share data via state (INCORRECT!)."""
    
    def __init__(self, name):
        super().__init__()
        self.name = name
    
    async def process(self, context):
        print(f"🔍 {self.name}:")
        print(f"  My state keys: {list(context.state.keys())}")
        
        # INCORRECT: Trying to add data to state for other nodes
        context.state[f'{self.name}_timestamp'] = time.time()
        context.state[f'{self.name}_processed'] = True
        
        return {'node': self.name, 'attempt': 'state_sharing'}

class StateReader(Node):
    """Tries to read state from previous nodes (INCORRECT!)."""
    
    async def process(self, context):
        print(f"\\n📊 StateReader trying to read shared state:")
        print(f"  My state keys: {list(context.state.keys())}")
        
        # Try to count processed nodes (won't work!)
        processed_nodes = [k for k in context.state.keys() if k.endswith('_processed')]
        print(f"  ❌ Found {len(processed_nodes)} processed nodes (expected 3, got 0!)")
        print(f"  ❌ Previous nodes' state changes are NOT visible here!")
        
        return {'found_nodes': len(processed_nodes)}

# Create chain
sharer1 = StateSharer("Node1")
sharer2 = StateSharer("Node2")
sharer3 = StateSharer("Node3")
reader = StateReader()

sharer1 >> sharer2 >> sharer3 >> reader

# Run
graph1 = Graph(start=sharer1)
result1 = await graph1.run({'message': 'test'})

print(f"\\n🎯 Result: Only found {result1.content['found_nodes']} nodes")
print("❌ This pattern FAILS because state is node-local!\\n")


print("\\n" + "="*60)
print("=== Example 3B: CORRECT Pattern - Using Outputs ===" )
print("✅ This shows using outputs for linear data flow!\\n")

class DataCollector(Node):
    """Collects data correctly using outputs."""
    
    def __init__(self, name):
        super().__init__()
        self.name = name
    
    async def process(self, context):
        # Get data from previous node via inputs
        input_data = context.inputs.content
        processing_chain = input_data.get('processing_chain', [])
        
        print(f"📝 {self.name}:")
        print(f"  Received chain: {processing_chain}")
        
        # Use state for THIS node's own data (e.g., invocation count)
        if 'invocation_count' not in context.state:
            context.state['invocation_count'] = 0
        context.state['invocation_count'] += 1
        
        # CORRECT: Return data via outputs to share with next nodes
        new_chain = processing_chain + [{
            'node': self.name,
            'timestamp': time.time(),
            'invocations': context.state['invocation_count']
        }]
        
        return {
            'processing_chain': new_chain,
            'last_node': self.name
        }

class ChainAnalyzer(Node):
    """Analyzes the processing chain received via inputs."""
    
    async def process(self, context):
        # Get data from previous nodes via inputs (CORRECT!)
        input_data = context.inputs.content
        chain = input_data.get('processing_chain', [])
        
        print(f"\\n📊 ChainAnalyzer received data:")
        print(f"  ✅ Processing chain length: {len(chain)}")
        print(f"  ✅ Nodes that processed this: {[item['node'] for item in chain]}")
        
        return {
            'chain': chain,
            'total_nodes': len(chain),
            'analysis_complete': True
        }

# Create chain
collector1 = DataCollector("Collector1")
collector2 = DataCollector("Collector2")
collector3 = DataCollector("Collector3")
analyzer = ChainAnalyzer()

collector1 >> collector2 >> collector3 >> analyzer

# Run
graph2 = Graph(start=collector1)
result2 = await graph2.run({'processing_chain': []})

print(f"\\n🎯 Final Result:")
print(f"  ✅ Correctly tracked {result2.content['total_nodes']} nodes")
print(f"  ✅ Chain: {[item['node'] for item in result2.content['chain']]}")


print("\\n" + "="*60)
print("=== Example 3C: BEST Pattern - Using Graph State ===" )
print("✅ This shows using graph state for shared data!\\n")

class GraphStateCollector(Node):
    """Collects data using graph state for coordination."""
    
    def __init__(self, name):
        super().__init__()
        self.name = name
    
    async def process(self, context):
        # Use node state for THIS node's own data
        if 'invocation_count' not in context.state:
            context.state['invocation_count'] = 0
        context.state['invocation_count'] += 1
        
        # Use graph state for shared coordination data
        processing_chain = await context.graph_state.get('processing_chain', [])
        
        print(f"📝 {self.name}:")
        print(f"  Graph state chain: {processing_chain}")
        
        # Append to the shared processing chain
        processing_chain.append({
            'node': self.name,
            'timestamp': time.time(),
            'invocations': context.state['invocation_count']
        })
        
        await context.graph_state.set('processing_chain', processing_chain)
        
        # Increment shared counter
        counter = await context.graph_state.get('counter', 0)
        await context.graph_state.set('counter', counter + 1)
        
        # Return minimal data via outputs (just pass through signal)
        return {'processed': True, 'node': self.name}

class GraphStateAnalyzer(Node):
    """Analyzes data from graph state."""
    
    async def process(self, context):
        # Access shared graph state directly
        chain = await context.graph_state.get('processing_chain', [])
        counter = await context.graph_state.get('counter', 0)
        
        print(f"\\n📊 GraphStateAnalyzer accessing graph state:")
        print(f"  ✅ Processing chain length: {len(chain)}")
        print(f"  ✅ Nodes that processed this: {[item['node'] for item in chain]}")
        print(f"  ✅ Total counter: {counter}")
        
        return {
            'total_nodes': len(chain),
            'total_counter': counter,
            'analysis_complete': True
        }

# Create chain
gs_collector1 = GraphStateCollector("GSCollector1")
gs_collector2 = GraphStateCollector("GSCollector2")
gs_collector3 = GraphStateCollector("GSCollector3")
gs_analyzer = GraphStateAnalyzer()

gs_collector1 >> gs_collector2 >> gs_collector3 >> gs_analyzer

# Run with initial graph state
graph3 = Graph(start=gs_collector1, initial_state={'processing_chain': [], 'counter': 0})
result3 = await graph3.run({'message': 'test'})

print(f"\\n🎯 Final Result:")
print(f"  ✅ Correctly tracked {result3.content['total_nodes']} nodes using graph state")
print(f"  ✅ Total counter: {result3.content['total_counter']}")

# Show final graph state
final_chain = await graph3.get_state('processing_chain')
print(f"  ✅ Final chain from graph state: {[item['node'] for item in final_chain]}")


print("\\n" + "="*60)
print("=== Key Takeaway ===" )
print("❌ DON'T: Try to share data via context.state - it's node-local")
print("✅ DO: Use outputs to pass transformed data to next nodes")
print("✅ DO: Use graph state for shared counters, flags, and coordination")
print("✅ DO: Use node state only for a node's own persistent data")

## Example 4: Data Transformation Pipeline

Let's build a practical data transformation pipeline that processes structured data.

In [ ]:
class DataValidator(Node):
    """Validates input data structure."""
    
    async def process(self, context):
        data = context.inputs.content
        
        print(f"✅ Validating data: {data}")
        
        # Validate required fields
        required_fields = ['users']
        missing_fields = [field for field in required_fields if field not in data]
        
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
        
        users = data.get('users', [])
        
        return {
            'original_data': data,
            'user_count': len(users),
            'validation_passed': True,
            'users': users
        }

class DataEnricher(Node):
    """Enriches data with additional information."""
    
    async def process(self, context):
        data = context.inputs.content
        users = data.get('users', [])
        
        print(f"🔧 Enriching data for {len(users)} users...")
        
        # Enrich each user with computed fields
        enriched_users = []
        for user in users:
            enriched_user = user.copy()
            
            # Add computed fields
            name = user.get('name', '')
            enriched_user['name_length'] = len(name)
            enriched_user['name_uppercase'] = name.upper()
            enriched_user['initials'] = ''.join([n[0].upper() for n in name.split() if n])
            enriched_user['has_email'] = '@' in user.get('email', '')
            
            enriched_users.append(enriched_user)
        
        result = data.copy()
        result['enriched_users'] = enriched_users
        result['enrichment_complete'] = True
        
        return result

class DataAggregator(Node):
    """Aggregates and summarizes data."""
    
    async def process(self, context):
        data = context.inputs.content
        users = data.get('enriched_users', [])
        
        print(f"📊 Aggregating data from {len(users)} users...")
        
        # Calculate aggregates
        total_name_length = sum(user.get('name_length', 0) for user in users)
        users_with_email = sum(1 for user in users if user.get('has_email', False))
        
        # Group by initial
        initials_count = {}
        for user in users:
            initial = user.get('initials', '')
            if initial:
                initials_count[initial] = initials_count.get(initial, 0) + 1
        
        summary = {
            'total_users': len(users),
            'total_name_length': total_name_length,
            'average_name_length': total_name_length / len(users) if users else 0,
            'users_with_email': users_with_email,
            'email_percentage': (users_with_email / len(users) * 100) if users else 0,
            'initials_distribution': initials_count
        }
        
        result = data.copy()
        result['summary'] = summary
        result['aggregation_complete'] = True
        
        return result

class DataExporter(Node):
    """Exports data to final format."""
    
    async def process(self, context):
        data = context.inputs.content
        summary = data.get('summary', {})
        
        print("📤 Exporting data...")
        
        # Create export format
        export_data = {
            'export_timestamp': time.time(),
            'summary': summary,
            'metadata': {
                'total_nodes_processed': 4,
                'data_pipeline': 'validation -> enrichment -> aggregation -> export'
            }
        }
        
        # Print formatted summary
        print("\n📋 Data Processing Summary:")
        print(f"  Total Users: {summary.get('total_users', 0)}")
        print(f"  Average Name Length: {summary.get('average_name_length', 0):.1f}")
        print(f"  Users with Email: {summary.get('users_with_email', 0)} ({summary.get('email_percentage', 0):.1f}%)")
        print(f"  Initials Distribution: {summary.get('initials_distribution', {})}")
        
        return export_data

# Sample data
sample_users = [
    {'name': 'Alice Johnson', 'email': 'alice@example.com'},
    {'name': 'Bob Smith', 'email': 'bob@example.com'},
    {'name': 'Charlie Brown', 'email': ''},
    {'name': 'Diana Prince', 'email': 'diana@themyscira.com'},
    {'name': 'Eve Wilson', 'email': 'eve@example.org'}
]

# Create pipeline
validator = DataValidator()
enricher = DataEnricher()
aggregator = DataAggregator()
exporter = DataExporter()

# Connect nodes
validator >> enricher >> aggregator >> exporter

# Create graph
data_pipeline = Graph(start=validator)

print("=== Example 4: Data Transformation Pipeline ===")
print(f"Processing {len(sample_users)} users through pipeline...\n")

# Run pipeline
result = await data_pipeline.run({'users': sample_users})

print(f"\n🎯 Pipeline completed successfully!")
print(f"Export created at: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(result.content['export_timestamp']))}")

## Key Concepts Deep Dive

### Understanding Data Flow and Storage (CRITICAL!)

**The Golden Rule: Three Storage Types for Three Different Needs**

Spark provides three distinct mechanisms for data storage, each serving a specific purpose:

1. **Outputs**: Flow from one node to its direct successors
2. **Node State**: Private to each node, persists across its invocations
3. **Graph State**: Shared across all nodes, accessible everywhere

### The Three Storage Types in Detail

#### 1. Outputs - Sequential Data Flow

**Purpose**: Pass transformed data from one node to the next in the pipeline

**How it works**:
- Node returns a dict from `process()`
- Return value becomes next node's `context.inputs.content`
- Data flows linearly through the graph edges

**When to use**:
- Each node transforms or adds to the data
- Next node needs the result of previous node
- Data flow follows the graph structure

```python
# Node 1
async def process(self, context):
    data = context.inputs.content
    result = transform(data)
    return {'result': result}  # Flows to next node

# Node 2 receives it
async def process(self, context):
    previous_result = context.inputs.content['result']  # From Node 1
```

#### 2. Node State - Private Persistent Data

**Purpose**: Store data that's specific to ONE node across multiple invocations

**How it works**:
- Accessed via `context.state` (a dict-like object)
- Persists only for that node
- Other nodes CANNOT see it

**When to use**:
- Tracking invocation counts
- Caching computed values
- Retry attempts or internal state
- Node-specific configuration

```python
async def process(self, context):
    # Track this node's own invocation count
    if 'count' not in context.state:
        context.state['count'] = 0
    context.state['count'] += 1
    
    # Use it for node-specific logic
    if context.state['count'] > 5:
        print("This node has been called 5+ times")
    
    return {'invocations': context.state['count']}
```

#### 3. Graph State - Globally Shared Data

**Purpose**: Store data that ALL nodes in the graph need to access or modify

**How it works**:
- Accessed via `context.graph_state` (async methods)
- Shared across ALL nodes in the graph
- Thread-safe with automatic locking in concurrent mode
- Persists across multiple `graph.run()` calls

**When to use**:
- Shared counters (total items processed across all nodes)
- Accumulated results (collecting outputs from multiple branches)
- Coordination flags (should_stop, is_ready, etc.)
- Global configuration that all nodes need

```python
async def process(self, context):
    # Read shared counter
    counter = await context.graph_state.get('total_items', 0)
    
    # Update it
    await context.graph_state.set('total_items', counter + 1)
    
    # Append to shared list
    results = await context.graph_state.get('all_results', [])
    results.append(my_result)
    await context.graph_state.set('all_results', results)
    
    # Or use transaction for atomic updates
    async with context.graph_state.transaction() as state:
        state['x'] = state.get('x', 0) + 1
        state['y'] = state.get('y', 0) + 1
    
    return {'done': True}
```

### Comparison Table

| Feature | Outputs | Node State | Graph State |
|---------|---------|------------|-------------|
| **Scope** | Next node(s) only | Single node only | All nodes |
| **Lifetime** | One execution | Across invocations | Across invocations & runs |
| **Access** | `context.inputs.content` | `context.state` | `context.graph_state` |
| **Flow** | Follows graph edges | Doesn't flow | Available everywhere |
| **Thread-safe** | N/A | Yes | Yes (auto-locking) |
| **Use for** | Transformed data | Node-specific state | Coordination & sharing |

### Decision Tree: Which Storage Type to Use?

```
Do multiple non-adjacent nodes need this data?
├─ YES → Use Graph State
└─ NO
   └─ Does the next node in the pipeline need this data?
      ├─ YES → Use Outputs
      └─ NO
         └─ Does only THIS node need it across invocations?
            ├─ YES → Use Node State
            └─ NO → Don't store it
```

### Common Patterns

#### Pattern 1: Pipeline Transformation (Use Outputs)
```python
# Each node transforms and passes data forward
input_node >> processor >> enricher >> output_node
```

#### Pattern 2: Shared Counter (Use Graph State)
```python
# Multiple nodes increment a shared counter
async def process(self, context):
    counter = await context.graph_state.get('counter', 0)
    await context.graph_state.set('counter', counter + 1)
    return {'done': True}
```

#### Pattern 3: Node Invocation Tracking (Use Node State)
```python
# Track how many times THIS node has run
async def process(self, context):
    context.state['runs'] = context.state.get('runs', 0) + 1
    return {'run_count': context.state['runs']}
```

### Graph State Initialization and Access

```python
# Initialize graph with state
graph = Graph(
    start=start_node,
    initial_state={'counter': 0, 'results': [], 'config': {...}}
)

# Run the graph
result = await graph.run()

# Access state after execution
counter = await graph.get_state('counter')
all_state = graph.get_state_snapshot()

# Reset for next run
graph.reset_state({'counter': 0})
```

### ExecutionContext Structure

```python
class ExecutionContext:
    inputs: NodeMessage           # Data from previous node (FLOWS via outputs)
    state: NodeState              # This node's own state (PRIVATE, persists)
    graph_state: GraphState       # Shared state (GLOBAL, accessible to all)
    metadata: ExecutionMetadata   # Execution timing/attempts
    outputs: Any                  # Current node's output (becomes next inputs)
```

**Remember**:
- `context.inputs` = data from parent node's outputs (flows one direction)
- `context.state` = this node's private state (stays local, persists)
- `context.graph_state` = shared across all nodes (global, persists)
- Return value = outputs that flow to successor nodes

## Hands-On Exercises

### Exercise 1: Create a 3-Node Calculator Pipeline

Create a pipeline that:
1. Takes two numbers as input
2. Calculates sum, difference, and product
3. Formats the results as a string

```python
# Your code here
class NumberInput(Node):
    async def process(self, context):
        # Implement this
        pass

class Calculator(Node):
    async def process(self, context):
        # Implement this
        pass

class Formatter(Node):
    async def process(self, context):
        # Implement this
        pass
```

In [ ]:
# Exercise 1 Solution
class NumberInput(Node):
    """Inputs two numbers for calculation."""
    
    async def process(self, context):
        inputs = context.inputs.content
        num1 = inputs.get('num1', 5)
        num2 = inputs.get('num2', 3)
        
        print(f"🔢 Input numbers: {num1} and {num2}")
        
        return {
            'number1': num1,
            'number2': num2,
            'operation': 'basic_arithmetic'
        }

class Calculator(Node):
    """Performs calculations on two numbers."""
    
    async def process(self, context):
        data = context.inputs.content
        num1 = data.get('number1', 0)
        num2 = data.get('number2', 0)
        
        print(f"🧮 Calculating operations for {num1} and {num2}...")
        
        calculations = {
            'sum': num1 + num2,
            'difference': num1 - num2,
            'product': num1 * num2,
            'quotient': num1 / num2 if num2 != 0 else 'undefined',
            'power': num1 ** num2
        }
        
        result = data.copy()
        result['calculations'] = calculations
        
        return result

class Formatter(Node):
    """Formats calculation results."""
    
    async def process(self, context):
        data = context.inputs.content
        num1 = data.get('number1', 0)
        num2 = data.get('number2', 0)
        calc = data.get('calculations', {})
        
        print("📝 Formatting results...")
        
        formatted_results = f"""
=== Calculation Results ===
Numbers: {num1} and {num2}
Sum: {num1} + {num2} = {calc.get('sum', 'N/A')}
Difference: {num1} - {num2} = {calc.get('difference', 'N/A')}
Product: {num1} × {num2} = {calc.get('product', 'N/A')}
Quotient: {num1} ÷ {num2} = {calc.get('quotient', 'N/A')}
Power: {num1}^{num2} = {calc.get('power', 'N/A')}
============================
        """.strip()
        
        print(formatted_results)
        
        return {
            'formatted_output': formatted_results,
            'calculation_complete': True
        }

# Create and test the calculator pipeline
input_node = NumberInput()
calc_node = Calculator()
format_node = Formatter()

# Connect nodes
input_node >> calc_node >> format_node

# Create graph
calculator_graph = Graph(start=input_node)

print("=== Exercise 1: Calculator Pipeline ===")
result = await calculator_graph.run({'num1': 12, 'num2': 4})

print(f"\n✅ Calculator pipeline completed!")

### Exercise 2: Build a Text Processing Workflow

Create a workflow that processes text through multiple steps:
1. Input text
2. Count words and characters
3. Find the longest word
4. Generate a summary

In [ ]:
# Exercise 2 Solution
class TextInput(Node):
    """Inputs text for processing."""
    
    async def process(self, context):
        text = context.inputs.content.get('text', 'The quick brown fox jumps over the lazy dog')
        
        print(f"📝 Input text: '{text}'")
        
        return {
            'original_text': text,
            'processing_step': 'input_complete'
        }

class TextCounter(Node):
    """Counts words and characters."""
    
    async def process(self, context):
        data = context.inputs.content
        text = data.get('original_text', '')
        
        print("🔢 Counting words and characters...")
        
        words = text.split()
        characters = len(text)
        characters_no_spaces = len(text.replace(' ', ''))
        
        result = data.copy()
        result.update({
            'word_count': len(words),
            'character_count': characters,
            'character_count_no_spaces': characters_no_spaces,
            'words': words
        })
        
        return result

class WordAnalyzer(Node):
    """Finds longest word and analyzes vocabulary."""
    
    async def process(self, context):
        data = context.inputs.content
        words = data.get('words', [])
        
        print("🔍 Analyzing words...")
        
        if words:
            longest_word = max(words, key=len)
            shortest_word = min(words, key=len)
            average_word_length = sum(len(word) for word in words) / len(words)
            
            # Count unique words
            unique_words = set(word.lower() for word in words)
        else:
            longest_word = shortest_word = ''
            average_word_length = 0
            unique_words = set()
        
        result = data.copy()
        result.update({
            'longest_word': longest_word,
            'shortest_word': shortest_word,
            'average_word_length': round(average_word_length, 2),
            'unique_word_count': len(unique_words),
            'vocabulary': list(unique_words)
        })
        
        return result

class SummaryGenerator(Node):
    """Generates a final summary."""
    
    async def process(self, context):
        data = context.inputs.content
        
        print("📊 Generating summary...")
        
        summary = f"""
=== Text Analysis Summary ===
Original Text: {data.get('original_text', 'N/A')}
Words: {data.get('word_count', 0)}
Characters (with spaces): {data.get('character_count', 0)}
Characters (no spaces): {data.get('character_count_no_spaces', 0)}
Longest Word: {data.get('longest_word', 'N/A')}
Shortest Word: {data.get('shortest_word', 'N/A')}
Average Word Length: {data.get('average_word_length', 0)}
Unique Words: {data.get('unique_word_count', 0)}
============================
        """.strip()
        
        print(summary)
        
        return {
            'summary': summary,
            'analysis_complete': True,
            'processing_steps': ['input', 'counting', 'analysis', 'summary']
        }

# Create text processing pipeline
text_input = TextInput()
text_counter = TextCounter()
word_analyzer = WordAnalyzer()
summary_gen = SummaryGenerator()

# Connect nodes
text_input >> text_counter >> word_analyzer >> summary_gen

# Create graph
text_pipeline = Graph(start=text_input)

print("=== Exercise 2: Text Processing Workflow ===")
test_text = "Spark makes building AI workflows easy and powerful"
result = await text_pipeline.run({'text': test_text})

print(f"\n✅ Text processing completed!")
print(f"Processing steps: {result.content['processing_steps']}")

### Exercise 3: Experiment with Data Transformations

Create your own pipeline that transforms data in interesting ways. Be creative!

In [ ]:
# Exercise 3: Create Your Own Pipeline
# Example: Number Transformation Pipeline

class NumberGenerator(Node):
    """Generates a sequence of numbers."""
    
    async def process(self, context):
        start = context.inputs.content.get('start', 1)
        end = context.inputs.content.get('end', 10)
        
        numbers = list(range(start, end + 1))
        print(f"🔢 Generated numbers: {numbers}")
        
        return {'numbers': numbers, 'range': f"{start}-{end}"}

class NumberTransformer(Node):
    """Transforms numbers in various ways."""
    
    async def process(self, context):
        numbers = context.inputs.content.get('numbers', [])
        
        transformations = {
            'squares': [n**2 for n in numbers],
            'cubes': [n**3 for n in numbers],
            'doubles': [n*2 for n in numbers],
            'evens': [n for n in numbers if n % 2 == 0],
            'odds': [n for n in numbers if n % 2 != 0]
        }
        
        print("🔄 Applied transformations:")
        for name, values in transformations.items():
            print(f"  {name}: {values}")

        return {
            'numbers': numbers,
            'transformations': transformations
        }


class StatisticsCalculator(Node):
    """Calculates statistics on transformed data."""
    
    async def process(self, context):
        inputs = context.inputs.content
        numbers = inputs.get('numbers', [])
        transformations = inputs.get('transformations', {})
        
        stats = {
            'original_sum': sum(numbers),
            'original_avg': sum(numbers) / len(numbers) if numbers else 0,
            'squares_sum': sum(transformations.get('squares', [])),
            'even_count': len(transformations.get('evens', [])),
            'odd_count': len(transformations.get('odds', []))
        }
        
        print(f"📊 Statistics: {stats}")
        
        result = dict(inputs)
        result['statistics'] = stats
        return result

# Create your custom pipeline
generator = NumberGenerator()
transformer = NumberTransformer()
stats_calc = StatisticsCalculator()

# Connect your nodes
generator >> transformer >> stats_calc

# Create graph
custom_pipeline = Graph(start=generator)

print("=== Exercise 3: Custom Number Transformation Pipeline ===")
result = await custom_pipeline.run({'start': 3, 'end': 8})

print(f"\n✅ Custom pipeline completed!")
print(f"Final statistics: {result.content['statistics']}")

# Now try modifying this pipeline or creating your own!
print("\n💡 Try creating your own pipeline with different transformations!")

## Summary & Next Steps

### What You Learned

- ✅ **Graph Fundamentals**: How to create and connect nodes into flows
- ✅ **Edge Creation**: Using the `>>` operator to connect nodes
- ✅ **Auto-Discovery**: How Spark automatically discovers connected nodes
- ✅ **ExecutionContext**: Understanding how data flows between nodes
- ✅ **Data Passing**: Using `context.inputs` and return values
- ✅ **Three Storage Types**: CRITICAL distinction between outputs, node state, and graph state
- ✅ **Graph State**: Using globally shared state for coordination across nodes
- ✅ **Pipeline Building**: Creating multi-step data processing workflows

### Key Concepts Mastered

- Node creation and the `process()` method
- Graph construction with auto-discovery
- Data flow patterns in Spark
- ExecutionContext structure and usage
- **The Actor Model: Three distinct storage mechanisms**
  - Outputs for sequential data flow
  - Node state for private persistence
  - Graph state for global coordination
- Building practical data pipelines
- Error handling in node processing
- Correct state usage for different scenarios

### The Most Important Lesson

**Three Storage Types - Choose the Right One!**

```python
# ❌ WRONG: Node state doesn't flow between nodes
context.state['for_next_node'] = data

# ✅ RIGHT: Outputs flow to next nodes
return {'for_next_node': data}

# ✅ RIGHT: Graph state is accessible everywhere
await context.graph_state.set('shared_counter', 42)

# ✅ RIGHT: Use node state for node's own data only
context.state['my_invocation_count'] += 1
```

### Decision Guide: Which Storage Type?

**Use Outputs when:**
- Data flows linearly through the pipeline
- Each node transforms the data for the next node
- The graph structure defines the data flow

**Use Node State when:**
- Tracking a node's own invocation count or internal state
- Caching results specific to one node
- Data should NOT be visible to other nodes

**Use Graph State when:**
- Multiple nodes need to coordinate (counters, flags)
- Accumulating results from different parts of the graph
- Sharing configuration or context across all nodes
- Nodes that aren't directly connected need to share data

### Common Patterns

```python
# Pattern 1: Simple data pipeline (outputs)
input_node >> processor_node >> output_node
graph = Graph(start_node=input_node)

# Pattern 2: Shared counter (graph state)
graph = Graph(start_node=start_node, initial_state={'counter': 0})
# In nodes: await context.graph_state.set('counter', counter + 1)

# Pattern 3: Node-specific tracking (node state)
# In node: context.state['invocations'] = context.state.get('invocations', 0) + 1

# Pattern 4: Data validation and enrichment (outputs)
validator >> enricher >> aggregator >> exporter

# Pattern 5: Multi-step transformation (outputs + graph state)
collector >> cleaner >> analyzer >> reporter
# Use outputs for transformed data, graph state for metrics
```

### Best Practices

1. **Single Responsibility**: Each node should do one thing well
2. **Clear Data Flow**: Make it obvious what data each node expects and returns
3. **Use Outputs for Sequential Flow**: When data goes node → next node
4. **Use Graph State for Coordination**: When multiple nodes need shared data
5. **Use Node State for Privacy**: When only that node needs the data
6. **Error Handling**: Always validate inputs in your nodes
7. **Descriptive Names**: Use clear node and variable names
8. **Initialize Graph State**: Set initial values when creating the graph

### Graph State Examples

```python
# Initialize with state
graph = Graph(
    start=start_node,
    initial_state={'counter': 0, 'results': [], 'is_ready': False}
)

# Run and access state
result = await graph.run()
counter = await graph.get_state('counter')

# Reset between runs
graph.reset_state({'counter': 0, 'results': []})
```

### Next Tutorial

In **Tutorial 4: Your First AI Agent**, you'll learn how to:
- Understand the difference between Nodes and Agents
- Create and configure an Agent
- Work with different LLM providers
- Handle agent inputs and outputs
- Use AgentConfig for agent configuration

### Continue Your Learning

- Try building pipelines for your own data processing needs
- Experiment with graph state for coordination patterns
- Practice with the example file: `e006_graph_state_counter.py`
- Think about when to use outputs vs. graph state
- Remember: **Choose the right storage type for the job!**

---

🎉 **Congratulations!** You've mastered the fundamentals of creating flows and graphs in Spark, including the critical distinction between the three storage types: outputs for sequential flow, node state for private data, and graph state for global coordination. This Actor Model foundation with proper isolation and sharing patterns is essential as we move on to working with AI agents in the next tutorial!